In [ ]:
import matplotlib.pyplot as plt
from utils import *
from scipy.integrate import solve_ivp
from numpy.linalg import inv
import tensorflow as tf
from dolfin import *
from ffc.fiatinterface import create_quadrature

In [ ]:
############################
# Model parameters
############################
method = "NM_ROM"
nu = 0.1
A = 0.5
filename = "output/burgers_1D/nu_" + str(nu) + "/"


In [ ]:
############################
# Read snapshots
############################
print("Reading mesh and solution of the full order model")
mesh, u_ref = read_mesh_and_function(filename + "FOM", "u")


In [ ]:
############################
# Perform AutoEncoding
# R: number of dofs in the full order model
# r: number of dofs in the reduced order model
############################

# This is the number of dofs in the reduced order model
n = 15

N, time_steps = u_ref.shape

# split test and train data
X_train, X_test, y_train, y_test = train_test_split(
    u_ref.T, u_ref.T, test_size=0.1, random_state=42)



# This is our input image
input_img = keras.Input(shape=(N, ))

# "encoded" is the encoded representation of the input
# encoded = layers.Dense(r, activation='sigmoid')(input_img)
encoded = layers.Normalization()(input_img)
encoded = layers.Dense(2000, activation='linear')(encoded)
encoded = layers.Dense(n, activation='linear')(encoded)


# "decoded" is the lossy reconstruction of the input
decoded = layers.Dense(N, activation='linear', kernel_constraint=tf.keras.constraints.NonNeg())(encoded)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)

# This model maps an input to its encoded representation
encoder = keras.Model(input_img, encoded)

# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(n,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))
# decoder = keras.Model(encoded_input, autoencoder.layers[-1](autoencoder.layers[-2](encoded_input)))

num_epochs = 10

# opt = tf.keras.optimizers.SGD(
#     learning_rate=0.01
# )
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

autoencoder.compile(optimizer=opt, loss='mean_squared_error')

class fewerPrintouts(tf.keras.callbacks.Callback):
    def __init__(self, interval):
        self.interval = interval

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.interval == 0:
            print('Epoch {}, loss = {}, val_loss = {}'.format(epoch, logs['loss'], logs['val_loss']))

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                              patience=10, min_lr=1e-6,verbose=1)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=200)

autoencoder.fit(X_train, X_train,
                epochs=num_epochs,
                batch_size=20,
                validation_data=(X_test, X_test),
                verbose=0,
                callbacks=[fewerPrintouts(10),early_stopping])

In [ ]:
############################
# Function space
############################
V = FunctionSpace(mesh, "CG", 1)

############################
# Functions
############################
u = Function(V)
u_old = Function(V)
u0 = Function(V)
v = TestFunction(V)

############################
# Initial condition
############################
u0_expr = Expression(
    "x[0] < 1 ? 1+A*(sin(2*pi*x[0]-pi/2)+1) : 1", degree=1, A=A)
u0.interpolate(u0_expr)


In [ ]:
############################
# Time control
############################
t_start = 0.0
t_final = 0.5
t_steps = 500
t_sequence = np.linspace(t_start, t_final, t_steps + 1)
dt = (t_final - t_start) / t_steps

In [ ]:
def decode(red_vec):
    return tf.reshape(decoder(tf.reshape(red_vec, [1,-1])), [N])

def encode(vec):
    return encoder(tf.reshape(vec, [1,-1]))

In [ ]:
qe = FiniteElement(
    family = "Quadrature",
    cell = mesh.ufl_cell(),
    degree = 2,
    quad_scheme="default"
    )
Q = FunctionSpace(mesh, qe)
Qe = Q.element()

@tf.function(jit_compile=True)
def objective(ur, ur_old):
    def residual_(v, grad_v, u, grad_u, u_old):
        return v*(u-u_old)/dt-nu*tf.tensordot(grad_v,grad_u,1)+v*grad_u[0]*u
    
    ut = decoder(ur)
    ut_old = decoder(ur_old)
    
    result = 0

    for e in range(mesh.num_cells()):
        cell = Cell(mesh, e)
        dofs = V.dofmap().cell_dofs(e)
        coordinates = cell.get_coordinate_dofs()
        ref_points, weights = create_quadrature(mesh.ufl_cell(), 2)
        qpoints = Q.element().tabulate_dof_coordinates(cell)
        ut_cell = tf.gather(ut[0],dofs)
        ut_old_cell = tf.gather(ut_old[0],dofs)
        
        for qp,weight in enumerate(weights):
            qpoint = qpoints[qp]
            v_ = V.element().evaluate_basis_all(qpoint, coordinates, cell.orientation()).astype('float32')
            grad_v_ = V.element().evaluate_basis_derivatives_all(1, qpoint, coordinates, cell.orientation()).reshape(2,2).astype('float32')
            u_qp = tf.tensordot(v_, ut_cell, 1)
            u_old_qp = tf.tensordot(v_, ut_old_cell, 1)
            grad_u_qp = tf.tensordot(grad_v_, ut_cell, 1)
            detJ = np.abs(coordinates[0]-coordinates[2])
            r_qp = residual_(v_[qp], grad_v_[qp], u_qp, grad_u_qp, u_old_qp)
            result = result + r_qp * detJ * weight
    
    return result

In [ ]:
def residual_and_jacobian(ur, ur_old):
    with tf.GradientTape() as t2:
        t2.watch(ur)
        with tf.GradientTape() as t1:
            t1.watch(ur)
            obj = objective(ur, ur_old)
        r = t1.gradient(obj, ur)
    J = t2.jacobian(r, ur)
    return r, J

In [ ]:
ur0 = encode(u0.vector().get_local())
ur = tf.identity(ur0)
r, J = residual_and_jacobian(ur, ur0)

In [ ]:
print(J)

In [ ]:
# r_form = (dot(v, (u - u_old) / Constant(dt)) - inner(nu * grad(u), grad(v)) + inner(u.dx(0) * u, v)) * dx
# dr_form_dD = derivative(r_form, u)

# def Jacobian(ur):
#     with tf.GradientTape() as g:
#         ur_reshaped = tf.reshape(ur, [1,-1])
#         g.watch(ur_reshaped)
#         D = decoder(ur_reshaped)
#         dD_dur = g.jacobian(D, ur_reshaped).numpy()[0,:,0,:]
#         D = tf.reshape(D, [N])

#     u.vector().set_local(D.numpy())
#     dr_dD = assemble(dr_form_dD).array()

#     J = np.matmul(dr_dD, dD_dur)
#     return J

In [ ]:
def solve(u, RTOL=1e-5, ATOL=1e-6):
    def orthogonalize(J, r):
        return np.matmul(J.T, r)

    def converged(r, r0):
        if np.linalg.norm(r) < ATOL:
            return True
        if np.linalg.norm(r) < RTOL * np.linalg.norm(r0):
            return True
        return False

    r = residual(u)
    J = Jacobian(u)

    r0 = orthogonalize(J, r)

    for itr in range(50):
        print('Itr = {:}, |R| = {:}'.format(itr, np.linalg.norm(orthogonalize(J,r))))

        if converged(orthogonalize(J, r), r0):
            return u
        
        du = - np.matmul(np.linalg.inv(np.matmul(J.T, J)), orthogonalize(J, r))
        u = u + du
        
        r = residual(u)
        J = Jacobian(u)

In [ ]:
ur0 = encode(u0.vector().get_local())
ur = tf.identity(ur0)

u_approx = np.zeros((N, time_steps))

for i, t in enumerate(t_sequence):
    print('step = {}, t = {}'.format(i, t))
    u_old.vector().set_local(decode(ur).numpy())
    ur = solve(ur,1e-3,1e-3)
    u_approx[:,i] = decode(ur).numpy()

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(u_approx, aspect='auto', cmap='jet', vmin=1, vmax=2)
cb = fig.colorbar(im)
# ax.set_xlabel("$t$")
# ax.set_ylabel("$x$")
# ax.set_xticks(np.linspace(0, t_steps, 3))
# ax.set_xticklabels(np.linspace(t_start, t_final, 3))
# ytick_loc = np.linspace(0, V.dim() - 1, 3).astype(int)
# ax.set_yticks(ytick_loc)
# ax.set_yticklabels(V.tabulate_dof_coordinates()[ytick_loc, 0])
# plt.tight_layout(pad=0)
# plt.savefig(filename + "ref.png")
plt.show()